In [36]:
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import pywt
import os
import numpy as np
from numpy import array
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESS_PATH = 'compressed_low'
DECOMPRESS_PATH = 'decompressed'
IMAGE_NAME = 'low.png'
MESH_KEYS = ['faces', 'neighbour', 'owner', 'points']
PTURHO_KEYS = ['p', 'T', 'U', 'rho']

class Decompressor:
    def __init__(self):
        if not os.path.exists(DECOMPRESS_PATH):
            os.mkdir(DECOMPRESS_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
    
    def total_decompress(self, compressed_data: str, q_step: np.float64, min_val: np.float64, signals) -> np.ndarray:
        z_decoded = self.entropy_compressor.decompress(bytes.fromhex(compressed_data))
        print('z decoded')
        dequantized = self.quantizator.dequantize(z_decoded, q_step, min_val)
        print('dequantized')
        decompressed = self.wavelet_compressor.inverse_wavelet_transform(dequantized)

        return decompressed

    def load_to_file(self, data: List[np.ndarray | np.float64], file_path: str, header_data: dict[str]) -> None:
        z_compressed, q_step, min_val = data
        decompressed = self.total_decompress(z_compressed, q_step, min_val)
        
    def is_two_floats(self, line: str) -> bool:
        digitcnt_dot_cnt = 0
        for t in line:
            if t.isdigit() or t == '.':
                digitcnt_dot_cnt += 1
        return len(line) - 1 == digitcnt_dot_cnt
    
    def read_mesh_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        decompressed = {}
        chunks = []
        current_chunk = {'q_step': None, 'min_val': None, 'data': []}

        with open(file_path, 'r') as file:
            for line in file:

                if ' ' in line and all(s.replace('.', '', 1).replace('-', '', 1).isdigit() for s in line.split()):
                    if current_chunk['q_step'] is not None:

                        chunks.append(current_chunk)
                        current_chunk = {'q_step': None, 'min_val': None, 'data': []}

                    q_step, min_val = map(float, line.split())
                    current_chunk['q_step'] = q_step
                    current_chunk['min_val'] = min_val
                else:
                    current_chunk['data'].append(line.strip())


            if current_chunk['q_step'] is not None:
                chunks.append(current_chunk)
        for i in range(4):
            chunk = chunks[i]
            q_step = np.float64(float(chunk['q_step']))
            min_val = np.float64(float(chunk['min_val']))
            hexvalue = chunk['data'][0]
            signal_dict = eval(''.join(chunk['data'][1:]))
            # decompressed[MESH_KEYS[i]] = [hexvalue, q_step, min_val, signal_dict]
            decompressed[MESH_KEYS[i]] = self.total_decompress(hexvalue, q_step, min_val, signal_dict)
        return decompressed
        

    def read_pturho_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        i = 0
        with open(file_path, mode='r') as f:
            times = f.readline().strip().split(';')
            for key in times:
                data[key] = None
            while True:
                quantilization_keys_p = f.readline().strip()
                z_compressed_p = f.readline().strip()
                details_p = eval(f.readline().strip())

                quantilization_keys_t = f.readline().strip()
                z_compressed_t = f.readline().strip()
                details_t = eval(f.readline().strip())

                quantilization_keys_u = f.readline().strip()
                z_compressed_u = f.readline().strip()
                details_u = eval(f.readline().strip())

                quantilization_keys_rho = f.readline().strip()
                z_compressed_rho = f.readline().strip()
                details_rho = eval(f.readline().strip())

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys_p or not z_compressed_p:
                    break
                
                q_step_p, min_val_p = map(float, quantilization_keys_p.split())
                q_step_t, min_val_t = map(float, quantilization_keys_t.split())
                q_step_u, min_val_u = map(float, quantilization_keys_u.split())
                q_step_rho, min_val_rho = map(float, quantilization_keys_rho.split())
                data[times[i]] = [
                    [z_compressed_p, q_step_p, min_val_p, details_p],
                    [z_compressed_t, q_step_t, min_val_t, details_t],
                    [z_compressed_u, q_step_u, min_val_u, details_u],
                    [z_compressed_rho, q_step_rho, min_val_rho, details_rho],
                ]
                i += 1
        return data

In [37]:
decompressor = Decompressor()
mesh_data = decompressor.read_mesh_file('compressed/mesh')
print('read')
decompressor.total_decompress(*mesh_data['faces'])

z decoded
dequantized


ValueError: Unexpected detail coefficient type: <class 'numpy.float64'>. Detail coefficients must be a dictionary of arrays as returned by wavedecn. If you are using pywt.array_to_coeffs or pywt.unravel_coeffs, please specify output_format='wavedecn'